In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SeqIO
import os
import re
from pyfaidx import Fasta
from scipy.stats import chi2_contingency, mannwhitneyu
from statsmodels.stats.multitest import multipletests

In [3]:
# SNP Enrichment
## Binomial Test
# def snp_enrich(vcf, cpg_bed, reference):
#     # Read in the VCF file
#     vcf = pd.read_csv(vcf,
#                     sep='\t',
#                     comment='#',
#                     names=['CHROM','POS','ID','REF','ALT',
#             'QUAL','FILTER','INFO','FORMAT',
#             'sample'])

#     # Keep only the SNPs, i.e. single bases in REF and ALT
#     vcf = vcf[(vcf['ALT'].str.len() == 1) & (vcf['REF'].str.len() == 1)]

#     cpgs = pd.read_csv(cpg_bed,
#                     sep='\t',
#                     names=['Chrom','Start','Stop','ID','Score','Strand'])

#     # Minimap2 identified a mutation at position 149,470 = G to A.
#     # There is a CpG site at position 149,468 to 149,470.
#     # Therefore, add 1 to the start position of the CpG site to find SNPs affecting
#     # the C and use the stop position to find SNPs affecting the G.
#     cpgs['C'] = cpgs['Start'] + 1
#     cpgs['G'] = cpgs['Stop']

#     vcf['SNP_hit_C'] = vcf['POS'].isin(cpgs['C'])
#     vcf['SNP_hit_G'] = vcf['POS'].isin(cpgs['G'])
#     vcf['DoubleCount'] = vcf['SNP_hit_C'] & vcf['SNP_hit_G']

#     # Get total number of SNPs before removing double counted SNPs
#     total_snps = vcf.shape[0]

#     # Drop the double counted SNPs
#     vcf = vcf[vcf['DoubleCount'] == False]

#     snp_prob = (total_snps / len(reference))
#     snps_hit_cpg = (vcf['SNP_hit_C'].sum() + vcf['SNP_hit_G'].sum())


#     # k = number of successes, E.g. number of SNPs that hit a CpG site.
#     # n = number of trials, E.g. number of SNPs.
#     # p = probability of success, E.g. probability of a SNP occurring anywhere.

#     k = snps_hit_cpg
#     n = total_snps
#     p = snp_prob

#     test = stats.binomtest(k = k, n = n, p = p, alternative='greater')

#     #return test.statistic, test.pvalue
#     return snps_hit_cpg, total_snps, len(reference)

#def snp_enrichment_freq(vcf, cpg_bed, reference):

# Create a dictionary to store the contingency table
# For each base, is it:
# 1. A SNP in a CpG site
# 2. A SNP not in a CpG site
# 3. Not a SNP in a CpG site
# 4. Not a SNP not in a CpG site
# contingency_table = {
#     'base_hit_snp_cpg'        :0,
#     'base_hit_snp_not_cpg'    :0,
#     'base_hit_cpg_not_snp'    :0,
#     'base_hit_not_cpg_not_snp':0
#     }
def chiSquare_SNPs(chrom, VCF, CPG_BED, REFERENCE):
    
    # Read in the VCF file
    vcf = pd.read_csv(VCF,
            sep='\t',
            comment='#',
            names=['CHROM','POS','ID','REF','ALT',
    'QUAL','FILTER','INFO','FORMAT',
    'sample'])

    # Keep only the SNPs, i.e. single bases in REF and ALT
    vcf = vcf[(vcf['ALT'].str.len() == 1) & (vcf['REF'].str.len() == 1)]

    # Read in CpG bed file
    cpgs = pd.read_csv(CPG_BED,
                sep='\t',
                names=['Chrom','Start','Stop','ID','Score','Strand'])

    # Minimap2 identified a mutation at position 149,470 = G to A.
    # There is a CpG site at position 149,468 to 149,470.
    # Therefore, add 1 to the start position of the CpG site to find SNPs affecting
    # the C and use the stop position to find SNPs affecting the G.
    cpgs['C'] = cpgs['Start'] + 1
    cpgs['G'] = cpgs['Stop']

    vcf['SNP_hit_C'] = vcf['POS'].isin(cpgs['C'])
    vcf['SNP_hit_G'] = vcf['POS'].isin(cpgs['G'])
    vcf['DoubleCount'] = vcf['SNP_hit_C'] & vcf['SNP_hit_G']

    # Drop the double counted SNPs
    vcf = vcf[vcf['DoubleCount'] == False]

    cpgs_snps = {}    # SNPs that hit a CpG site
    non_cpg_snps = {} # SNPs that do not hit a CpG site

    for row in vcf.itertuples():
        if row.SNP_hit_C == True or row.SNP_hit_G == True:
            chrpos = str(row.CHROM) + '_' + str(row.POS)
            cpgs_snps[chrpos] = None
        else:
            chrpos = str(row.CHROM) + '_' + str(row.POS)
            non_cpg_snps[chrpos] = None

    chrom_seq = REFERENCE[str(chrom)]

    is_cpg = []
    is_snp = []


    unaffected_cpgs = cpgs.shape[0] - len(cpgs_snps.keys())
    unaffected_cpgs_bases = unaffected_cpgs * 2
    affected_cpgs_bases = len(cpgs_snps.keys()) * 2
    non_cpgs_snps = len(non_cpg_snps.keys())
    remaining_bases = len(chrom_seq) - (unaffected_cpgs_bases + affected_cpgs_bases + non_cpgs_snps)

    # Add the is_cpg and is_snp labels to the list
    is_cpg.extend(['is_cpg'] * affected_cpgs_bases)
    is_snp.extend(['is_snp'] * affected_cpgs_bases)

    is_cpg.extend(['not_cpg'] * non_cpgs_snps)
    is_snp.extend(['is_snp'] * non_cpgs_snps)

    is_cpg.extend(['is_cpg'] * unaffected_cpgs_bases)
    is_snp.extend(['not_snp'] * unaffected_cpgs_bases)

    is_cpg.extend(['not_cpg'] * remaining_bases)
    is_snp.extend(['not_snp'] * remaining_bases)

    contingency_table = pd.DataFrame({'is_cpg':is_cpg, 'is_snp':is_snp})
    x = pd.crosstab(contingency_table['is_cpg'], contingency_table['is_snp'])
    x.to_csv(f'snp_fisherexact/{chrom}_contingency_table.csv', header=True, index=True)
    # print(x.values)

    _stat, _pval = chi2_contingency(x.values)[:2]
    
    return _stat, _pval
    


In [4]:
stats_ = []
pvals_ = []

REFERENCE = Fasta('/Users/callummacphillamy/PhD/methylation_chapter/'
                      'gigascience_revisions/references/brahman/Brahman.fa')

for chrom in tqdm(range(1, 30)):
    VCF = f'./brahman/vcf/brahman_{chrom}_vs_angus_{chrom}.vcf.gz'
    CPG_BED = ('/Users/callummacphillamy/PhD/methylation_chapter/'
               'gigascience_revisions/Brah-Ang_methylation/genome/Brahman_CpG/'
               f'beds/Brahman_{chrom}.CpGs.bed.gz')
    
    stat_, pval_, = chiSquare_SNPs(chrom, VCF, CPG_BED, REFERENCE)
    stats_.append(stat_)
    pvals_.append(pval_)


    


100%|██████████| 29/29 [08:27<00:00, 17.49s/it]


In [5]:
# Perform multiple testing correction
pvals_ = np.array(pvals_)
multipletests(pvals_, method='fdr_bh')

(array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 0.0017671710005495722,
 0.001724137931034483)

In [6]:
# Determine fold enrichment for CpG SNPs vs non-CpG SNPs
is_snp_is_cpg = 0 # A
is_snp_not_cpg = 0 # B
not_snp_is_cpg = 0 # C
not_snp_not_cpg = 0 # D

for i in tqdm(range(1,30)):
    cont_table = pd.read_csv(f'snp_chi/{i}_contingency_table.csv', index_col=0)
    is_snp_is_cpg += cont_table.loc['is_cpg', 'is_snp']
    is_snp_not_cpg += cont_table.loc['not_cpg', 'is_snp']
    not_snp_is_cpg += cont_table.loc['is_cpg', 'not_snp']
    not_snp_not_cpg += cont_table.loc['not_cpg', 'not_snp']
A_in_cat_of_interest = is_snp_is_cpg
total_cat_of_interest = is_snp_is_cpg + is_snp_not_cpg
B_in_background = not_snp_is_cpg
total_background = not_snp_is_cpg + not_snp_not_cpg
print('fold enrichment:', (A_in_cat_of_interest / total_cat_of_interest) / (B_in_background / total_background))

100%|██████████| 29/29 [00:00<00:00, 857.01it/s]

fold enrichment: 12.953283537771606


## SV CpG Counts

In [7]:
b_small_indels = []
b_svs = {}
b_sv_coords = {'chrom':[],
			   'start':[],
			   'stop':[]}
b_sv_len = {}

a_small_indels = []
a_svs = {}
a_sv_coords = {'chrom':[],
			   'start':[],
			   'stop':[]}
a_sv_len = {}

# As these are structural variants and indels greater than 1 bp in length, we
# can just count the occurrences of a CpG site in the sequence of each variant
for chrom in range(1,30):
	vcf = pd.read_csv(f'./brahman/vcf/brahman_{chrom}_vs_angus_{chrom}.vcf.gz',
				  sep='\t',
				  comment='#',
				  names=['CHROM','POS','ID','REF','ALT',
		   'QUAL','FILTER','INFO','FORMAT',
		   'sample'])
	a_svs[str(chrom)] = 0
	b_svs[str(chrom)] = 0
	a_sv_len[str(chrom)] = 0
	b_sv_len[str(chrom)] = 0
	for record in tqdm(vcf.itertuples()):
		if len(record.REF) == 1 and len(record.ALT) == 1:
			continue
		info = record.INFO.split(';')
		q_start = int(info[1].split('=')[1])
		if len(record.REF) <= 50 and len(record.ALT) <= 50:
			b_small_indels.append(len(re.findall('CG', record.REF))) 
			a_small_indels.append(len(re.findall('CG', record.ALT)))
		elif len(record.REF) <= 50 and len(record.ALT) > 50:
			b_small_indels.append(len(re.findall('CG', record.REF)))
			#a_svs.append(len(re.findall('CG', record.ALT)))
			a_svs[str(record.CHROM)] += len(re.findall('CG', record.ALT))
			a_sv_len[str(record.CHROM)] += len(record.ALT)
			a_sv_coords['chrom'].append(record.CHROM)
			a_sv_coords['start'].append(q_start - 2)
			a_sv_coords['stop'].append(q_start - 2 + len(record.ALT))
		elif len(record.REF) > 50 and len(record.ALT) <= 50:
			#b_svs.append(len(re.findall('CG', record.REF)))
			b_svs[str(record.CHROM)] += len(re.findall('CG', record.REF))
			b_sv_len[str(record.CHROM)] += len(record.REF)
			b_sv_coords['chrom'].append(record.CHROM)
			b_sv_coords['start'].append(record.POS - 1)
			b_sv_coords['stop'].append(record.POS - 1 + len(record.REF))
			a_small_indels.append(len(re.findall('CG', record.ALT)))
		elif len(record.REF) > 50 and len(record.ALT) > 50:
			#b_svs.append(len(re.findall('CG', record.REF)))
			b_svs[str(record.CHROM)] += len(re.findall('CG', record.REF))
			b_sv_coords['chrom'].append(record.CHROM)
			b_sv_coords['start'].append(record.POS - 1)
			b_sv_coords['stop'].append(record.POS - 1 + len(record.REF))
			b_sv_len[str(record.CHROM)] += len(record.REF)
			
			#a_svs.append(len(re.findall('CG', record.ALT)))
			a_svs[str(record.CHROM)] += len(re.findall('CG', record.ALT))
			a_sv_coords['chrom'].append(record.CHROM)
			a_sv_coords['start'].append(q_start - 2)
			a_sv_coords['stop'].append(q_start - 2 + len(record.ALT))
			a_sv_len[str(record.CHROM)] += len(record.ALT)
		else:
			print('Something unforeseen happened.')
			break

791895it [00:00, 994064.96it/s] 
681471it [00:00, 1000502.14it/s]
568905it [00:00, 945398.59it/s]
597400it [00:00, 1020763.86it/s]
543304it [00:00, 949149.68it/s]
606340it [00:00, 972080.42it/s]
567375it [00:00, 972804.81it/s]
518721it [00:00, 972196.01it/s]
532052it [00:00, 971289.79it/s]
537562it [00:00, 1013653.76it/s]
496830it [00:00, 1004110.71it/s]
418196it [00:00, 988201.11it/s]
360277it [00:00, 1008743.76it/s]
401396it [00:00, 1002322.98it/s]
473250it [00:00, 1009122.15it/s]
397779it [00:00, 1000223.65it/s]
358587it [00:00, 986152.69it/s]
250576it [00:00, 977988.05it/s]
325654it [00:00, 980055.70it/s]
240723it [00:00, 939743.80it/s]
363936it [00:00, 946057.57it/s]
298564it [00:00, 982746.12it/s]
299415it [00:00, 963904.57it/s]
367108it [00:00, 1009346.12it/s]
181793it [00:00, 977823.60it/s]
235475it [00:00, 991152.50it/s]
280249it [00:00, 988089.88it/s]
179962it [00:00, 990823.56it/s]
299971it [00:00, 1005665.15it/s]


In [8]:
print(f'CpGs in Brahman small indels: {np.sum(b_small_indels)}')
print(f'CpGs in Brahman SVs: {np.sum(b_svs)}')
print(f'CpGs in Angus small indels: {np.sum(a_small_indels)}')
print(f'CpGs in Angus SVs: {np.sum(a_svs)}')

CpGs in Brahman small indels: 29668
CpGs in Brahman SVs: {'1': 10085, '2': 7778, '3': 8112, '4': 7526, '5': 9504, '6': 8265, '7': 8307, '8': 9096, '9': 10847, '10': 7756, '11': 5571, '12': 4911, '13': 5518, '14': 4503, '15': 9186, '16': 5615, '17': 5191, '18': 6763, '19': 5980, '20': 3857, '21': 6151, '22': 3360, '23': 7257, '24': 6322, '25': 2466, '26': 3708, '27': 4733, '28': 2276, '29': 4746}
CpGs in Angus small indels: 29452
CpGs in Angus SVs: {'1': 11296, '2': 7837, '3': 8267, '4': 9580, '5': 8336, '6': 9466, '7': 8629, '8': 9635, '9': 7190, '10': 7315, '11': 5179, '12': 7021, '13': 4067, '14': 4858, '15': 7089, '16': 4539, '17': 5109, '18': 4262, '19': 3982, '20': 4386, '21': 4877, '22': 2872, '23': 6793, '24': 4945, '25': 2048, '26': 3637, '27': 4604, '28': 2421, '29': 6097}


In [5]:
# Save the coordinates of the SVs
pd.DataFrame.from_dict(a_sv_coords).sort_values(by=['chrom','start']).to_csv('angus_SV_coords.bed', sep='\t', index=False, header=False)
pd.DataFrame.from_dict(b_sv_coords).sort_values(by=['chrom','start']).to_csv('brahman_SV_coords.bed', sep='\t', index=False, header=False)

In [ ]:
## SHELL SCRIPT
# Get the complement of the SV regions
bedtools complement -i angus_SV_coords.bed -g Angus.chrom.sizes > angus_non_SV_coords.bed
bedtools complement -i brahman_SV_coords.bed -g Brahman.chrom.sizes > brahman_non_SV_coords.bed

# Get the fasta sequences of the complement regions
bedtools getfasta -fi /Users/callummacphillamy/PhD/gigascience_revisions/references/angus/Angus.fa -bed angus_non_SV_coords.bed -fo angus_non_SV_seq.fa

bedtools getfasta -fi /Users/callummacphillamy/PhD/gigascience_revisions/references/brahman/Brahman.fa -bed brahman_non_SV_coords.bed -fo brahman_non_SV_seq.fa

In [9]:
# Load the Brahman non-SV fasta file
brahman_non_SV = SeqIO.to_dict(SeqIO.parse('./brahman_non_SV_seq.fa', 'fasta'))

# Load the Angus non-SV fasta file
angus_non_SV = SeqIO.to_dict(SeqIO.parse('./angus_non_SV_seq.fa', 'fasta'))

In [10]:
brahman_non_SV_cpgs = {}
brahman_non_sv_len = {}
angus_non_SV_cpgs = {}
angus_non_sv_len = {}

for k, v in tqdm(brahman_non_SV.items()):
	chrom = v.id.split(':')[0]
	if chrom not in brahman_non_SV_cpgs:
		brahman_non_SV_cpgs[chrom] = 0
		brahman_non_sv_len[chrom] = 0
	brahman_non_SV_cpgs[chrom] += len(re.findall('CG', str(v.seq)))
	brahman_non_sv_len[chrom] += len(v.seq) - 1

for k, v in tqdm(angus_non_SV.items()):
	chrom = v.id.split(':')[0]
	if chrom not in angus_non_SV_cpgs:
		angus_non_SV_cpgs[chrom] = 0
		angus_non_sv_len[chrom] = 0
	angus_non_SV_cpgs[chrom] += len(re.findall('CG', str(v.seq)))
	angus_non_sv_len[chrom] += len(v.seq) - 1


100%|██████████| 15500/15500 [00:04<00:00, 3343.19it/s]


In [11]:
def CpG_SV_enrichment_mwu(cpg_sv_dict,
                      sv_len_dict,
                      non_sv_cpg_dict,
                      non_sv_len_dict):
    pvalues = []
    for chrom in tqdm(range(1,30)):
        cpg_proportion = (cpg_sv_dict[str(chrom)] / sv_len_dict[str(chrom)])
        non_sv_cpg_proportion = (non_sv_cpg_dict[str(chrom)] / non_sv_len_dict[str(chrom)])
        
        pval = mannwhitneyu(cpg_proportion, non_sv_cpg_proportion,
                            alternative='greater').pvalue
        pvalues.append(pval)
    return pvalues

brahman_pvalues = CpG_SV_enrichment_mwu(b_svs, b_sv_len, brahman_non_SV_cpgs, brahman_non_sv_len)
angus_pvalues = CpG_SV_enrichment_mwu(a_svs, a_sv_len, angus_non_SV_cpgs, angus_non_sv_len)

100%|██████████| 29/29 [00:00<00:00, 2195.69it/s]


In [14]:
cpg_sv_dict = b_svs
sv_len_dict = b_sv_len
non_sv_cpg_dict = brahman_non_SV_cpgs
non_sv_len_dict = brahman_non_sv_len
cpg_sv_proportions = []
non_sv_cpg_proportions = []
for chrom in tqdm(range(1,30)):
    cpg_proportion = (cpg_sv_dict[str(chrom)] / sv_len_dict[str(chrom)])
    non_sv_cpg_proportion = (non_sv_cpg_dict[str(chrom)] / non_sv_len_dict[str(chrom)])
    
    cpg_sv_proportions.append(cpg_proportion)
    non_sv_cpg_proportions.append(non_sv_cpg_proportion)

print(np.mean(cpg_sv_proportions))
print(np.mean(non_sv_cpg_proportions))
print(np.mean(cpg_sv_proportions) / np.mean(non_sv_cpg_proportions))
print(mannwhitneyu(cpg_sv_proportions, non_sv_cpg_proportions, alternative='greater'))

100%|██████████| 29/29 [00:00<00:00, 221154.21it/s]

0.012860810682802025
0.010913646684440219
1.1784155245870211
MannwhitneyuResult(statistic=662.0, pvalue=8.917394796484191e-05)
